In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import os
import webbrowser
from threading import Timer
import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options=[{'label': 'All Sites', 'value': 'ALL'},
                                            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}],
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                     min=0, max=10000, step=1000,
                                     marks={0: '0 (kg)', 2500: '2500 (kg)', 5000: '5000 (kg)', 7500: '7500 (kg)', 10000: '10000 (kg)'},
                                     value=[min_payload, max_payload]),
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    # if all data should be shown
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values="class", 
        names='Launch Site', 
        title='Overall success rate')
        return fig
    # filtered dataframe for one launch site
    else:
        # count success and failure for each launch site and store in df
        group_count = spacex_df.groupby(['Launch Site', 'class']).size().reset_index()
        group_count.columns = ["Launch Site", "class", "count"]

        # filter dataframe dependent on selected drop down
        filtered_gc = group_count[group_count['Launch Site'] == entered_site]

        # pie chart
        fig = px.pie(filtered_gc, values= "count", 
        names=["failure", "success"], 
        title=('Success rate for launch site ' + entered_site))
        return fig
        # return the outcomes piechart for a selected site


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='payload-slider', component_property='value'),
                Input(component_id='site-dropdown', component_property='value')])

def get_scatter_plot(slider_val, entered_site):
    # if all launch sites are selected
    # filter according to slider
    min = float(slider_val[0])
    max = float(slider_val[1])
    payload_current = spacex_df[(spacex_df['Payload Mass (kg)'] >= min) & (spacex_df['Payload Mass (kg)'] <= max)]
    filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min) & (spacex_df['Payload Mass (kg)'] <= max)]
    if entered_site == 'ALL':
        fig1 = px.scatter(payload_current, 
                    x="Payload Mass (kg)", y="class", color="Booster Version Category",
                    title=('Correlation between payload and success for all sites'))
        fig1.update_layout(
                    # adjust yaxis to array and tick text  to the text genrated
                    yaxis={"tickmode":"array","tickvals": [0, 1], "ticktext": ["failure", "success"],
                           "title": "Success / failure of booster landing"})
        return fig1
    # if a specific launch site is selected
    else:
        fig1 = px.scatter(filtered_df, 
                    x="Payload Mass (kg)", y="class", color="Booster Version Category",
                    title=('Correlation between payload and success for launch site ' + entered_site))
        fig1.update_layout(
                    # adjust yaxis to array and tick text  to the text genrated
                    yaxis={"tickmode":"array","tickvals": [0, 1], "ticktext": ["failure", "success"]})

        return fig1


# start webserver and run app
def open_browser():
    if not os.environ.get("WERKZEUG_RUN_MAIN"):
        webbrowser.open_new('http://127.0.0.1:8050/')


if __name__ == "__main__":
    Timer(1, open_browser).start()
    app.run_server(debug=True, port=8050)